<a href="https://colab.research.google.com/github/taiga0803/q-macro-assignment/blob/main/qmacro_pj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
# pwtから最新データ(vol10.1)の読み込み
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

#必要な国とデータの抽出
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'pop', 'avh', 'rkna', 'emp','rtfpna', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()
data = data.sort_values('year')

In [49]:
# 必要な変数の計算
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha'])) # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha'])) # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N
data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()

# Calculate growth rates and shares
data['g'] = grouped_data['y_n_shifted'].diff() *100  # GDP per capitaの成長率（前年比）
data['tfp_growth'] = grouped_data['tfp_term_shifted'].diff() *100 # TFPの成長率（前年比）
data['capital_deepening'] = grouped_data['cap_term_shifted'].diff() *100 # 資本深化の成長率（前年比）
data['tfp_share'] = (data['tfp_growth'] / data['g'])*100   # GDP成長率に対するTFP成長率の寄与率（割合）
data['capital_share'] = (data['capital_deepening'] / data['g'])*100  # GDP成長率に対する資本深化の寄与率（割合）

# Remove missing values
data = data.dropna()

summary = data.groupby('countrycode').agg({'g': 'mean', 'tfp_growth': 'mean', 'capital_deepening': 'mean', 'tfp_share': 'mean', 'capital_share': 'mean'})

# Print output
print(summary)

                     g  tfp_growth  capital_deepening   tfp_share  \
countrycode                                                         
CAN          50.415497  -90.208130       -1619.285645  180.716080   
DEU         -15.100708 -159.566040         652.729614 -136.206543   
FRA          28.414307 -109.923546        1158.504028  275.135315   
GBR          56.980591  -60.276184        5223.901367  159.587021   
ITA         -74.203331 -167.333527        1108.676147  960.976685   
JPN         -17.607727 -153.087311        5987.361816   58.726482   
USA         -68.788300 -165.405731        3305.912598  209.647308   

             capital_share  
countrycode                 
CAN           -2285.085205  
DEU            2896.203613  
FRA             110.163406  
GBR           -6038.313477  
ITA           -1957.532471  
JPN           -4767.951172  
USA           -2845.221191  


<ipython-input-49-dc5f5864d272>:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
